# Train a 🐸 STT model with Common Voice data 💫

👋 Hello and welcome to Coqui (🐸) STT 

This notebook shows a **typical workflow** for **training** and **testing** an 🐸 STT model on data from Common Voice.

In this notebook, we will:

1. Download Common Voice data (pre-formatted for 🐸 STT)
2. Configure the training and testing runs
3. Train a new model
4. Test the model and display its performance

So, let's jump right in!

*PS - If you just want a working, off-the-shelf model, check out the [🐸 Model Zoo](https://www.coqui.ai/models)*

In [ ]:
## Install Coqui STT
! pip install -U pip
! pip install coqui_stt_training
## Install opus tools
! apt-get install libopusfile0 libopus-dev libopusfile-dev

## ✅ Download & format sample data for Serbian

**First things first**: we need some data.

We're training a Speech-to-Text model, so we want _speech_ and we want _text_. Specificially, we want _transcribed speech_. Let's download some audio and transcripts.

To focus on model training, we formatted the Common Voice data for you already, and you will find CSV files for `{train,test,dev}.csv` in the data directory.

Let's download some data for Serbian 😊


In [ ]:
### Download pre-formatted Common Voice data
import os
import tarfile
from coqui_stt_training.util.downloader import maybe_download

def download_preformatted_data():
    if not os.path.exists("serbian/sr-data"):
        maybe_download("sr-data.tar", "serbian/", "https://coqui-ai-public-data.s3.amazonaws.com/cv/7.0/sr-data.tar")
        print('\nExtracting data...')
        tar = tarfile.open("serbian/sr-data.tar", mode="r:")
        tar.extractall("serbian/")
        tar.close()
        print('\nFinished extracting data...')
    else:
        print('Found data - not extracting.')

# Download + extract Common Voice data
download_preformatted_data()

### 👀 Take a look at the data

In [ ]:
! ls serbian/sr-data
! wc -l serbian/sr-data/*.csv

## ✅ Configure & set hyperparameters

Coqui STT comes with a long list of hyperparameters you can tweak. We've set default values, but you will often want to set your own. You can use `initialize_globals_from_args()` to do this. 

You must **always** configure the paths to your data, and you must **always** configure your alphabet. Additionally, here we show how you can specify the size of hidden layers (`n_hidden`), the number of epochs to train for (`epochs`), and to initialize a new model from scratch (`load_train="init"`).

If you're training on a GPU, you can uncomment the (larger) training batch sizes for faster training.

In [ ]:
from coqui_stt_training.util.config import initialize_globals_from_args

initialize_globals_from_args(
    train_files=["serbian/sr-data/train.csv"],
    dev_files=["serbian/sr-data/dev.csv"],
    test_files=["serbian/sr-data/test.csv"],
    checkpoint_dir="serbian/checkpoints/",
    load_train="init",
    n_hidden=200,
    epochs=1,
    beam_width=1,
    #train_batch_size=128,
    #dev_batch_size=128,
    #test_batch_size=128,
)

### 👀 View all config settings

In [ ]:
from coqui_stt_training.util.config import Config

print(Config.to_json())

## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀 (using the configure you set above).

In [ ]:
from coqui_stt_training.train import train

train()

## ✅ Test the model

We made it! 🙌

Let's kick off the testing run, which displays performance metrics.

The settings we used here are for demonstration purposes, so you don't want to deploy this model into production. In this notebook we're focusing on the workflow itself, so it's forgivable 😇

You can still train a more State-of-the-Art model by finding better hyperparameters, so go for it 💪

In [ ]:
from coqui_stt_training.evaluate import test

test()